# Linear regression

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [10]:
from itertools import product

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear
from tqdm.auto import trange

from system_identification.lsqr_model import LeastSquaresModel

In [3]:
data = xr.open_dataset("data_smoothed.nc")
data

<xarray.Dataset>
Dimensions:                 (t: 10001, dim_0: 4, dim_1: 4, x_idx: 4, training_samples: 8000, inputs: 2, m: 1, outputs: 1, evaluation_samples: 2001, d0: 2, d1: 2)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx, training_samples, inputs, m, outputs, evaluation_samples, d0, d1
Data variables: (12/40)
    p_k1k1                  (t, dim_0, dim_1) float64 ...
    p_kk1                   (t, dim_0, dim_1) float64 ...
    phi                     (t, dim_0, dim_1) float64 ...
    gamma                   (t, dim_0, dim_1) float64 ...
    ps                      (t, dim_0, dim_1) float64 ...
    x_k1k1                  (t, x_idx) float64 ...
    ...                      ...
    alpha_estimate          (t) float64 ...
    training_inputs         (training_samples, inputs, m) float64 ...
    training_outputs        (training_samples, outputs, m) float64 ...
    evaluation_inputs       (evaluation_samples, inputs, m) float64 ...
    evaluation_outputs      (evaluation_samples, outputs, m) float64 ...
    input_range             (d0, d1) float64 ...

In [17]:
order_list = []
error_list = []

for order in trange(1, 21):
    model = LeastSquaresModel.new_polynomial(
        n_inputs=2,
        input_range=data.input_range.values,
        order=order)
    model

    model.train(
        inputs=data.training_inputs.values,
        reference_outputs=data.training_outputs.values,
        evaluation_inputs=data.evaluation_inputs.values,
        evaluation_reference_outputs=data.evaluation_outputs.values,
    )
    
    order_list.append(order)
    error_list.append(model.training_log.error.item())

  0%|          | 0/14 [00:00<?, ?it/s]

In [18]:
plt.figure()
plt.plot(order_list, error_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig = plt.figure()

reference_inputs = data.evaluation_inputs.values
reference_outputs = data.evaluation_outputs.values

output = model.evaluate(reference_inputs)
ax = fig.add_subplot(projection='3d')
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], output.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …